In [1]:
import numpy as np
from collections import defaultdict
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import colorsys

In [2]:
def probability_sum_equals(s, n, d):
    if n == 0:
        return 1 if s == 0 else 0
    if s < n or s > n * d:
        return 0
    # Use dynamic programming to calculate the probability
    dp = np.zeros((n + 1, s + 1))
    dp[0][0] = 1
    for i in range(1, n + 1):
        for j in range(i, min(s, i * d) + 1):
            dp[i][j] = sum(dp[i - 1][j - k] for k in range(1, d + 1) if j - k >= 0)
    return dp[n][s] / (d ** n)

In [3]:
class Player:
    """Base player class with strategy methods"""

    def __init__(self, id, strategy="random"):
        self.id = id
        self.strategy = strategy
        self.color = f"#{np.random.randint(0, 0xFFFFFF):06x}"
    
    def choose_starting_location(self, available_locations):
        """Choose initial settlement location"""
        if self.strategy == "random":
            return np.random.choice(available_locations)
        elif self.strategy == "greedy":
            # Choose location with highest resource probability
            return self._choose_best_location(available_locations)#, state, game_config)
        else:
            return np.random.choice(available_locations)

    def _choose_best_location(self, available_locations):
        """Helper to choose location with best resource probability"""
        
        best_score = -1
        best_location = available_locations[0]
        
        for location in available_locations:
            score = state['starting_point_attractiveness'].get(location, 0)
            if score > best_score:
                best_score = score
                best_location = location
        
        return best_location
    
    def choose_action(self, state, inventories, can_afford_func, dev_cards, game_config):
        """Choose what action to take during turn"""
        possible_actions = ["road", "settlement", "city", "development_card", "play_knight", "end_turn"]
        
        # Prioritize based on affordability and strategy
        priorities = []
        
        # Building priorities
        if can_afford_func(self.player_id, "settlement"):
            priorities.append("settlement")
        if can_afford_func(self.player_id, "city"):
            priorities.append("city")
        if can_afford_func(self.player_id, "road"):
            priorities.append("road")
        if can_afford_func(self.player_id, "development_card"):
            priorities.append("development_card")
        
        # Development card priorities
        if dev_cards[self.player_id]["knight"] > 0:
            # Play knight if we're close to largest army or need to move robber
            knights_played = state.get('knights_played', {})
            if knights_played.get(self.player_id, 0) >= 2 or random.random() < 0.3:
                priorities.insert(0, "play_knight")  # High priority
        
        # Strategy-based decision making
        if self.strategy == "aggressive" and priorities:
            # Aggressive players prefer building
            building_actions = [a for a in priorities if a in ["settlement", "city", "road"]]
            if building_actions:
                return random.choice(building_actions)
        
        # 70% chance to do something if possible, 30% chance to end turn
        if priorities and random.random() < 0.7:
            return random.choice(priorities)
        else:
            return "end_turn"
    
    def choose_trade(self, state, inventories, building_costs, get_trade_ratio_func):
        """Choose whether and what to trade"""
        inv = inventories[self.player_id]
        
        # Find what we need for priority builds
        for build_type in ["settlement", "city", "development_card", "road"]:
            cost = building_costs[build_type]
            missing = {}
            
            for res, needed in cost.items():
                if inv.get(res, 0) < needed:
                    missing[res] = needed - inv.get(res, 0)
            
            if missing:
                # Try to trade excess resources
                excess_resources = {res: qty for res, qty in inv.items() if qty > 3}
                
                for give_res, qty in excess_resources.items():
                    ratio = get_trade_ratio_func(self.player_id, give_res, state)
                    if qty >= ratio:
                        get_res = random.choice(list(missing.keys()))
                        return (give_res, get_res)
        
        return None
    
    def choose_robber_location(self, current_robber, valid_locations, state):
        """Choose where to place the robber"""
        if not valid_locations:
            return None
        
        if self.strategy == "aggressive":
            # Target locations with most opponent buildings
            best_location = None
            max_opponents = 0
            
            for location in valid_locations:
                neighboring_points = state['locations_to_neighboring_points'].get(
                    tuple(map(float, location)), set()
                )
                opponent_count = sum(1 for point in neighboring_points 
                                   if point in state['point_owner'] 
                                   and state['point_owner'][point] != self.player_id)
                
                if opponent_count > max_opponents:
                    max_opponents = opponent_count
                    best_location = location
            
            return best_location if best_location else random.choice(valid_locations)
        else:
            return random.choice(valid_locations)
    
    def choose_steal_target(self, adjacent_players, inventories):
        """Choose which player to steal from"""
        if not adjacent_players:
            return None
        
        if self.strategy == "aggressive":
            # Target player with most cards
            return max(adjacent_players, 
                      key=lambda p: sum(inventories[p].values()))
        else:
            return random.choice(list(adjacent_players))
    
    def choose_discard_cards(self, inventory, num_to_discard):
        """Choose which cards to discard when robber is rolled"""
        cards_list = []
        for resource, quantity in inventory.items():
            cards_list.extend([resource] * quantity)
        
        if len(cards_list) <= num_to_discard:
            return cards_list
        
        if self.strategy == "greedy":
            # Keep more valuable resources (wheat, ore)
            priority = {'wheat': 0, 'ore': 1, 'sheep': 2, 'wood': 3, 'brick': 4}
            cards_list.sort(key=lambda x: priority.get(x, 5), reverse=True)
            return cards_list[:num_to_discard]
        else:
            return random.sample(cards_list, num_to_discard)

In [4]:
resources = {0: 'sheep', 1: 'wood', 2: 'wheat', 3: 'brick', 4: 'ore', 5: 'desert'}

def plot_state_and_index(grid, game_data):
    """
    grid: 3D array [rows, cols, 2] where [:,:,0] is roll numbers and [:,:,1] is resources
    game_data: dict containing player_colors, point_owner, edge_owner, etc.
    """
    
    point_owner = game_data.get("point_owner", {})
    edge_owner = game_data.get("edge_owner", {})
    players = game_data.get("players", {})
    ports = game_data.get("ports", {})

    roll_grid = grid[:, :, 0]
    resource_grid = grid[:, :, 1]
    nrows, ncols = roll_grid.shape

    # Create hex locations with offset for even rows
    locations = np.argwhere(np.ones_like(roll_grid)).astype(float)
    even_rows = locations[:, 0] % 2 == 1
    locations[even_rows, 1] += 0.5

    point_index = {}
    points = []
    edge_index = {}
    edges = []
    point_to_edges = defaultdict(set)

    def normalize_point(pt, precision=6):
        """Round point coordinates to avoid float mismatch."""
        return (round(pt[0], precision), round(pt[1], precision))

    def get_point_id(pt):
        if pt not in point_index:
            point_index[pt] = len(points)
            points.append(pt)
        return point_index[pt]

    def get_edge_id(p1, p2):
        e = tuple(sorted((p1, p2)))
        if e not in edge_index:
            edge_index[e] = len(edges)
            edges.append(e)
            point_to_edges[e[0]].add(edge_index[e])
            point_to_edges[e[1]].add(edge_index[e])
        return edge_index[e]

    # Create the plot
    try:
        plt.figure(figsize=(ncols*2, nrows*2))
        plotted_lines = set()
        locations_to_neighboring_points = defaultdict(set)
        points_to_nodes = defaultdict(set)

        # Plot hexes and build adjacency structure
        for idx, point in enumerate(locations):
            x, y = point[1], point[0]  # Swap for plotting
            
            # Try to load resource image, fallback to colored circle if not found
            try:
                resource_code = int(grid[int(y), int(x), 1])
                resource_name = resources.get(resource_code, "unknown")
                img = mpimg.imread(f"resource_pngs/{resource_name}.png")
                
                size = .65
                img = img[::-1, :, :]  # Flip vertically
                plt.imshow(img, extent=[x - size/2, x + size/2, y - size/2, y + size/2], 
                          zorder=1, origin='upper')
            except:
                # Fallback: draw colored circle for resource
                resource_colors = {0: 'lightgreen', 1: 'brown', 2: 'saddlebrown',
                                   3: 'wheat', 4: 'red'}
                resource_code = int(grid[int(y), int(x), 1])
                color = resource_colors.get(resource_code, 'white')
                circle = plt.Circle((x, y), 0.4, color=color, alpha=0.7, zorder=1)
                plt.gca().add_patch(circle)
            
            # Add roll number
            roll_num = int(roll_grid[int(y), int(x)])
            plt.text(x, y, str(roll_num), color='white', fontsize=12, 
                    zorder=2, ha='center', va='center', weight='bold')
            
            # Add location coordinates (optional, for debugging)
            # plt.text(x-0.13, y-0.15, f"({int(x)}, {int(y)})", 
            #         fontsize=8, zorder=2, color="black")

            # Define hex edges coordinates
            edges_coords = [
                ((x+0.5, y-1/3), (x+0.5, y+1/3)),
                ((x+0.5, y+1/3), (x, y+2/3)),
                ((x, y+2/3), (x-0.5, y+1/3)),
                ((x-0.5, y+1/3), (x-0.5, y-1/3)),
                ((x-0.5, y-1/3), (x, y-2/3)),
                ((x, y-2/3), (x+0.5, y-1/3)),
            ]

            # Process each edge of the hexagon
            for p1, p2 in edges_coords:
                p1 = normalize_point(p1)
                p2 = normalize_point(p2)
                pid1 = get_point_id(p1)
                pid2 = get_point_id(p2)
                eid = get_edge_id(pid1, pid2)

                # Map this hex location to its neighboring points
                point_key = tuple(map(float, point))
                locations_to_neighboring_points[point_key].add(pid1)
                locations_to_neighboring_points[point_key].add(pid2)
                
                # Only add to points_to_nodes if the point is not a whole number (i.e., has a .5 in x or y)
                # If the point is on an even row (i.e., x or y has a .5), map to the corresponding integer node
                # Use integer-rounded-down coordinates for point_key to avoid float mismatch
                point_key = (int(np.floor(point[0])), int(np.floor(point[1])))
                points_to_nodes[pid1].add(point_key)
                points_to_nodes[pid2].add(point_key)
                # Determine edge color based on owner
                edge_color = 'orange'
                if edge_owner and eid in edge_owner:
                    player = edge_owner[eid]
                    edge_color = p_color = players[player].color

                # Plot edge only once
                if eid not in plotted_lines:
                    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], '-', 
                            color=edge_color, alpha=0.7, linewidth=2, zorder=3)
                    plotted_lines.add(eid)

        # Plot settlements and cities
        if point_owner:
            for pid, player in point_owner.items():
                if pid < len(points):
                    px, py = points[pid]
                    p_color = players[player].color

                    # Check if it's a city (larger marker)
                    marker_size = 100 if game_data.get('cities', {}).get(pid) == player else 50
                    marker = 's' if game_data.get('cities', {}).get(pid) == player else 'o'
                    
                    plt.scatter(px, py, c=p_color, s=marker_size, 
                              marker=marker, zorder=4, alpha=1, edgecolors='black')

        # Generate and plot ports
        ports = {}
        prob_of_general_port = 0.5
        np.random.seed(42)  # For consistent port generation
        
        for point_id, edge_set in point_to_edges.items():
            # Points with exactly 2 edges are on the boundary (potential ports)
            if len(edge_set) == 2 and point_id < len(points):
                # Choose port type
                port_options = list(resources.values()) + ['generic']
                port_weights = [(1-prob_of_general_port)/len(resources)] * len(resources) + [prob_of_general_port]
                trade_resource = np.random.choice(port_options, p=port_weights)
                
                ports[point_id] = trade_resource
                x, y = points[point_id]
                
                # Draw port indicator
                plt.scatter(x, y, marker='D', s=120, color='gray', 
                           alpha=0.7, zorder=2, edgecolors='black')
                
                # Add port text
                if trade_resource == 'generic':
                    plt.text(x, y-0.3, "3:1", fontsize=8, ha='center', 
                            color="black", weight='bold', zorder=5)
                else:
                    plt.text(x, y-0.3, f"2:1\n{trade_resource[:4]}", fontsize=6, 
                            ha='center', color="black", weight='bold', zorder=5)

        plt.gca().invert_yaxis()
        plt.axis('equal')
        plt.title("Settlers of Catan Board", fontsize=16)
        
        legend_elements = [plt.scatter([], [], c=player.color, s=100, label=f'Player {player.id}') 
                            for player in players]
        plt.legend(handles=legend_elements, loc='upper right')
        
        plt.tight_layout()
        
        # Only show plot if running interactively
        try:
            if plt.get_backend() != 'Agg':
                plt.show()
        except:
            pass  # Don't show if no display available
            
    except Exception as e:
        print(f"Plotting failed: {e}")
        # Continue without plotting

    # Calculate player ports (ports owned by each player)
    player_ports = defaultdict(set)
    if point_owner and ports:
        for point_id, owner in point_owner.items():
            if point_id in ports:
                port_resource = ports[point_id]
                player_ports[owner].add(port_resource)

    # Build player positions
    player_positions = defaultdict(set)
    if point_owner:
        for pos, player in point_owner.items():
            player_positions[player].add(pos)

    longest_road_info = game_data.get('longest_road_info', (None, 0))
    largest_army_info = game_data.get('largest_army_info', (None, 0))

    # Return comprehensive game state
    return {
        "grid": grid,
        "players": players,
        "points": points,
        "edges": edges,
        "point_to_edges": dict(point_to_edges),
        "point_index": point_index,
        "edge_index": edge_index,
        "locations_to_neighboring_points": dict(locations_to_neighboring_points),
        "points_to_nodes": points_to_nodes,
        "ports": ports,
        "point_owner": point_owner,
        "edge_owner": edge_owner,
        "player_ports": dict(player_ports),
        "player_positions": dict(player_positions),
        "player_scores": game_data.get('player_scores', {}),
        "cities": game_data.get('cities', {}),
        "robber_location": game_data.get('robber_location', None),
        "longest_road_info": longest_road_info,
        "largest_army_info": largest_army_info,
    }

In [5]:
def game_loop(game_config=None, seed=None):
    # Game parameters
    num_players = game_config.num_players if game_config else 3
    players = [Player(id=i) for i in range(num_players)]
    game_shape = game_config.board_size if game_config else (5, 5)
    n_die = game_config.n_die if game_config else 2
    die_sides = game_config.die_sides if game_config else 6
    max_turns = game_config.max_turns if game_config else 100
    victory_points_to_win = game_config.victory_points_to_win if game_config else 10

    max_roll = n_die * die_sides
    
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    # Create roll grid (excluding 7 as per original logic)
    roll_grid = np.random.choice(
        np.concatenate([np.arange(n_die, 7), np.arange(8, n_die * die_sides + 1)]),
        size=game_shape,
        p=[1/(max_roll-n_die) for _ in range(max_roll-n_die)]
    )
    
    # Resources
    # Use the global resources dictionary
    # Remove 'desert' from resources for resource_grid generation
    resource_keys = [k for k, v in resources.items() if v != 'desert']
    resource_grid = np.random.choice(
        resource_keys,
        size=game_shape,
        p=[1/len(resource_keys)] * len(resource_keys)
    )

    roll_grid[game_shape[0] // 2, game_shape[1] // 2] = 7
    resource_grid[game_shape[0] // 2, game_shape[1] // 2] = 5

    # Create lookup dictionaries
    resource_locations = {resource: set(zip(*np.where(resource_grid == resource))) for resource in resources}
    roll_locations = {roll: set(zip(*np.where(roll_grid == roll))) for roll in range(n_die, n_die * die_sides + 1)}
    
    grid = np.stack((roll_grid, resource_grid), axis=-1)
    
    # Building costs
    building_costs = {
        "road": {"wood": 1, "brick": 1},
        "settlement": {"wood": 1, "brick": 1, "sheep": 1, "wheat": 1},
        "city": {"wheat": 2, "ore": 3},
        "development_card": {"sheep": 1, "wheat": 1, "ore": 1}
    }
    
    # Development cards
    development_cards = {
        "knight": 14,
        "victory_point": 5,
        "road_building": 2,
        "year_of_plenty": 2,
        "monopoly": 2
    }
    
    # Create development card deck
    dev_card_deck = []
    for card_type, count in development_cards.items():
        dev_card_deck.extend([card_type] * count)
    random.shuffle(dev_card_deck)
    
    # Initialize player inventories and development cards
    inventories = {player: {res: 0 for res in resources.values()} for player in range(num_players)}
    player_dev_cards = {player: defaultdict(int) for player in range(num_players)}
    knights_played = {player: 0 for player in range(num_players)}

    # Give starting resources
    for player_id, player in enumerate(players):
        for res in resources.values():
            inventories[player_id][res] = 2  # Start with 2 of each resource
    
    # Initialize game state
    m, n = roll_grid.shape
    num_vertices = (2*m+1) * (n + 1) + n - 1#(m + 1) * (n + 1)
    
    #for player in players:

    point_owner = {}
    player_positions = defaultdict(set)

    remaining = set(range(num_vertices))
    for player_id, player in enumerate(players):
        player_start = player.choose_starting_location(list(remaining), state, game_config)
        point_owner[player_start] = player_id
        player_positions[player].add(player_start)
        remaining.discard(player_start)

    # Second placements
    for player_id, player in enumerate(players[::-1]):
        player_id = num_players - 1 - player_id
        player_second = player.choose_second_location(list(remaining), state, game_config)
        point_owner[player_second] = player_id
        player_positions[player].add(player_second)
        remaining.discard(player_second)

    """# Starting placements
    starting_locations = np.random.choice(num_vertices, size=players, replace=False)
    remaining = list(set(range(num_vertices)) - set(starting_locations))
    second_locations = np.random.choice(remaining, size=players, replace=False)
    
    # Player colors and initial setup
    player_colors = {i: f"#{np.random.randint(0, 0xFFFFFF):06x}" for i in range(players)}
    
    point_owner = {}
    # First placement (player 0 → player N-1)
    for player, loc in enumerate(starting_locations):
        point_owner[loc] = player
    
    # Second placement (player N-1 → player 0)
    for idx, loc in enumerate(second_locations[::-1]):
        player = players - 1 - idx
        point_owner[loc] = player
    
    # Player positions and scores
    player_positions = defaultdict(set)
    for loc, player in point_owner.items():
        player_positions[player].add(loc)"""

    player_scores = {player: 2 for player in range(num_players)}

    # Special achievements
    longest_road_info = (None, 0)  # (player, length)
    largest_army_info = (None, 0)  # (player, size)

    # Bundle game data
    game_data = {
        "players": players,
        "point_owner": point_owner,
        "edge_owner": {},
        "player_positions": player_positions,
        "player_scores": player_scores,
        "cities": {},  # point_id -> player (upgraded settlements)
        "longest_road_info": longest_road_info,
        "largest_army_info": largest_army_info,
    }
    
    # Initialize state
    state = plot_state_and_index(grid, game_data)
    
    # Helper functions
    def roll():
        rolls = np.random.randint(1, die_sides + 1, n_die)
        return int(np.sum(rolls))
    
    def is_game_over(state, turn):
        return turn > max_turns or any(calculate_total_score(player, state) >= 10 for player in range(num_players))
    
    def calculate_longest_road(player, state):
        """Calculate the longest road for a player using DFS"""
        if not state.get("edge_owner"):
            return 0
        
        # Get all edges owned by the player
        player_edges = {eid for eid, owner in state["edge_owner"].items() if owner == player}
        
        if len(player_edges) < 5:  # Need at least 5 roads to win longest road
            return 0
        
        # Build adjacency graph of player's roads
        road_graph = defaultdict(set)
        for edge_id in player_edges:
            p1, p2 = state["edges"][edge_id]
            road_graph[p1].add(p2)
            road_graph[p2].add(p1)
        
        def dfs_longest_path(start, visited, current_length):
            """DFS to find longest path from a starting point"""
            max_length = current_length
            
            for neighbor in road_graph[start]:
                if neighbor not in visited:
                    # Check if we can continue through this point
                    # Road is blocked if opponent has a settlement/city here
                    if neighbor in state.get("point_owner", {}) and state["point_owner"][neighbor] != player:
                        continue
                    
                    visited.add(neighbor)
                    length = dfs_longest_path(neighbor, visited, current_length + 1)
                    max_length = max(max_length, length)
                    visited.remove(neighbor)
            
            return max_length
        
        # Try starting from each endpoint or branch point
        max_road_length = 0
        for start_point in road_graph:
            visited = {start_point}
            length = dfs_longest_path(start_point, visited, 0)
            max_road_length = max(max_road_length, length)
        return max_road_length

    def update_longest_road(state):
        """Update longest road achievement"""
        road_lengths = {}
        for player in range(num_players):
            road_lengths[player] = calculate_longest_road(player, state)

        # Find player with longest road (minimum 5 roads)
        max_length = max(road_lengths.values())
        if max_length >= 5:
            current_longest = state.get("longest_road_info", (None, 0))[0]
            candidates = [p for p, length in road_lengths.items() if length == max_length]
            
            if len(candidates) == 1:
                new_longest = candidates[0]
                if current_longest != new_longest:
                    # Transfer longest road
                    if current_longest is not None:
                        state["player_scores"][current_longest] -= 2
                    state["player_scores"][new_longest] += 2
                    state["longest_road_info"] = (new_longest, max_length)
                    print(f"🛣️ Player {new_longest} now has the longest road ({max_length} segments)!")
                else:
                    # Same player still has it, just update the length
                    state["longest_road_info"] = (new_longest, max_length)
            else:
                # Multiple players tied for longest road
                if current_longest is None:
                    # No current holder and there's a tie - no one gets it
                    state["longest_road_info"] = (None, max_length)
                elif current_longest in candidates:
                    # Current holder is among tied players - they keep it
                    state["longest_road_info"] = (current_longest, max_length)
                else:
                    # Current holder is not among tied players - remove it, no new holder
                    state["player_scores"][current_longest] -= 2
                    state["longest_road_info"] = (None, max_length)
        else:
            # No one qualifies for longest road
            if state.get("longest_road_info", (None, 0))[0] is not None:
                current_longest, _ = state["longest_road_info"]
                state["player_scores"][current_longest] -= 2
                state["longest_road_info"] = (None, 0)
                print(f"🛣️ Player {current_longest} lost the longest road (no longer qualifies)")


    def update_largest_army(state):
        """Update largest army achievement"""
        # Find player with most knights (minimum 3)
        if not knights_played or max(knights_played.values()) == 0:
            max_knights = 0
        else:
            max_knights = max(knights_played.values())
            
        if max_knights >= 3:
            current_largest = state.get("largest_army_info", (None, 0))[0]
            candidates = [p for p, count in knights_played.items() if count == max_knights]
            
            if len(candidates) == 1:
                new_largest = candidates[0]
                if current_largest != new_largest:
                    # Transfer largest army
                    if current_largest is not None:
                        state["player_scores"][current_largest] -= 2
                    state["player_scores"][new_largest] += 2
                    state["largest_army_info"] = (new_largest, max_knights)
                    print(f"⚔️ Player {new_largest} now has the largest army ({max_knights} knights)!")
                else:
                    # Same player still has it, just update the count
                    state["largest_army_info"] = (new_largest, max_knights)
            else:
                # Multiple players tied for largest army
                if current_largest is None:
                    # No current holder and there's a tie - no one gets it
                    state["largest_army_info"] = (None, max_knights)
                elif current_largest in candidates:
                    # Current holder is among tied players - they keep it
                    state["largest_army_info"] = (current_largest, max_knights)
                else:
                    # Current holder is not among tied players - remove it, no new holder
                    state["player_scores"][current_largest] -= 2
                    state["largest_army_info"] = (None, max_knights)
                    print(f"⚔️ Player {current_largest} lost the largest army (tied with others)")
        else:
            # No one qualifies for largest army
            if state.get("largest_army_info", (None, 0))[0] is not None:
                current_largest, _ = state["largest_army_info"]
                state["player_scores"][current_largest] -= 2
                state["largest_army_info"] = (None, 0)
                print(f"⚔️ Player {current_largest} lost the largest army (no longer qualifies)")

    def calculate_total_score(player, state):
        """Calculate total victory points for a player - FIXED VERSION"""
        # Calculate from actual buildings and achievements
        all_buildings = state.get('player_positions', {}).get(player, set())
        cities = [pt for pt in all_buildings if pt in state.get('cities', {})]
        settlements = [pt for pt in all_buildings if pt not in state.get('cities', {})]
        
        building_points = len(settlements) + len(cities) * 2  # settlements: 1pt, cities: 2pts
        vp_card_points = player_dev_cards[player]['victory_point']
        longest_road_points = 2 if state.get("longest_road_info", (None, 0))[0] == player else 0
        largest_army_points = 2 if state.get("largest_army_info", (None, 0))[0] == player else 0
        
        total = building_points + vp_card_points + longest_road_points + largest_army_points
        return total
        
    def play_knight_card(player, state):
        """Play a knight development card"""
        if player_dev_cards[player]["knight"] > 0:
            player_dev_cards[player]["knight"] -= 1
            knights_played[player] += 1
            
            # Move robber and steal
            handle_robber(player, state)
            
            # Check for largest army
            print(f"Player {player} played a Knight card (total: {knights_played[player]})")
            
            update_largest_army(state)
            return True
        return False
    
    def distribute_resources(roll_result):
        """Distribute resources based on dice roll"""
        if roll_result == 7:
            return
            
        for location in roll_locations.get(roll_result, []):
            resource = resources[resource_grid[location]]
            loc_key = tuple(map(float, location))
            neighboring_points = state['locations_to_neighboring_points'].get(loc_key, set())
            
            for point in neighboring_points:
                if point in state['point_owner']:
                    player = state['point_owner'][point]
                    # Cities produce 2 resources, settlements produce 1
                    multiplier = 2 if point in state.get('cities', {}) else 1
                    inventories[player][resource] = inventories[player].get(resource, 0) + multiplier
    
    def handle_robber(player, state):
        """Handle robber placement and stealing"""
        # Get valid robber locations (not current robber position)
        current_robber = state.get('robber_location')
        all_locations = list(zip(*np.where(roll_grid != 7)))
        valid_locations = [loc for loc in all_locations if loc != current_robber]
        
        if valid_locations:
            robber_location = valid_locations[np.random.randint(len(valid_locations))]
            state['robber_location'] = robber_location
            print(f"Player {player} moved robber to {robber_location}")
            
            # Steal from a random adjacent player
            neighboring_points = state['locations_to_neighboring_points'].get(tuple(map(float, robber_location)), set())
            adjacent_players = set()
            for point in neighboring_points:
                if point in state['point_owner'] and state['point_owner'][point] != player:
                    adjacent_players.add(state['point_owner'][point])
            
            if adjacent_players:
                target = random.choice(list(adjacent_players))
                target_resources = [res for res, qty in inventories[target].items() if qty > 0]
                if target_resources:
                    stolen_resource = random.choice(target_resources)
                    inventories[target][stolen_resource] -= 1
                    inventories[player][stolen_resource] = inventories[player].get(stolen_resource, 0) + 1
                    print(f"Player {player} stole {stolen_resource} from Player {target}")
        else:
            print("No valid locations to place the robber.")
    
    def discard_excess_cards(player):
        """Handle discarding cards when 7 is rolled and player has >7 cards"""
        total_cards = sum(inventories[player].values())
        if total_cards > 7:
            to_discard = total_cards // 2
            print(f"Player {player} must discard {to_discard} cards")
            
            # Simple strategy: discard randomly
            cards_list = []
            for resource, quantity in inventories[player].items():
                cards_list.extend([resource] * quantity)
            
            discarded = random.sample(cards_list, min(to_discard, len(cards_list)))
            for resource in discarded:
                inventories[player][resource] -= 1
    
    def get_trade_ratio(player, resource_give, state):
        """Get bank trade ratio for a player"""
        player_ports = state["player_ports"].get(player, set())
        
        # Check for specific resource port (2:1)
        if resource_give in player_ports:
            return 2
        
        # Check for generic port (3:1)
        if "generic" in player_ports:
            return 3
        
        # Default bank rate (4:1)
        return 4
    
    def can_afford(player, build_type):
        """Check if player can afford a building"""
        cost = building_costs[build_type]
        inv = inventories[player]
        return all(inv.get(res, 0) >= qty for res, qty in cost.items())
    
    def pay_resources(player, build_type):
        """Deduct resources for a build"""
        cost = building_costs[build_type]
        for res, qty in cost.items():
            inventories[player][res] -= qty
    
    def execute_trade(player, resource_give, resource_get, state):
        """Execute a bank trade"""
        ratio = get_trade_ratio(player, resource_give, state)
        
        if inventories[player].get(resource_give, 0) >= ratio:
            inventories[player][resource_give] -= ratio
            inventories[player][resource_get] = inventories[player].get(resource_get, 0) + 1
            print(f"Player {player} traded {ratio} {resource_give} for 1 {resource_get}")
            return True
        return False
    
    def player_action(state, player_id, action):
        """Execute player building actions"""
        if action == "road":
            # Find buildable edges
            owned_points = [pt for pt, owner in state["point_owner"].items() if owner == player_id]
            owned_edges = set(state.get("edge_owner", {}).keys())
            
            possible_edges = set()
            for pt in owned_points:
                possible_edges.update(state["point_to_edges"].get(pt, set()))
            
            # Also include edges adjacent to owned roads
            for edge, owner in state.get("edge_owner", {}).items():
                if owner == player_id:
                    # Find edges connected to this road
                    edge_points = state["edges"][edge]
                    for point in edge_points:
                        possible_edges.update(state["point_to_edges"].get(point, set()))
            
            buildable_edges = list(possible_edges - owned_edges)
            
            if buildable_edges:
                edge = random.choice(buildable_edges)
                if "edge_owner" not in state:
                    state["edge_owner"] = {}
                state["edge_owner"][edge] = player_id
                print(f"Player {player_id} built a road on edge {edge}")
                return True
        
        elif action == "settlement":
            # Find valid settlement locations
            owned_edges = {e for e, owner in state.get("edge_owner", {}).items() if owner == player_id}
            candidate_points = set()
            
            for edge in owned_edges:
                edge_points = state["edges"][edge]
                candidate_points.update(edge_points)
            
            # Remove already owned points
            owned_points = set(state["point_owner"].keys())
            candidate_points -= owned_points
            
            # Check distance rule (no adjacent settlements)
            valid_points = []
            for pt in candidate_points:
                adjacent_edges = state["point_to_edges"].get(pt, set())
                adjacent_points = set()
                for edge in adjacent_edges:
                    adjacent_points.update(state["edges"][edge])
                adjacent_points.discard(pt)
                
                if not any(p in owned_points for p in adjacent_points):
                    valid_points.append(pt)
            
            if valid_points:
                point = random.choice(valid_points)
                state["point_owner"][point] = player_id
                state["player_positions"][player_id].add(point)
                state["player_scores"][player_id] += 1
                print(f"Player {player_id} built a settlement at point {point}")
                return True
        
        elif action == "city":
            # Upgrade existing settlement
            player_settlements = [pt for pt, owner in state["point_owner"].items() 
                                if owner == player_id and pt not in state.get("cities", {})]
            
            if player_settlements:
                point = random.choice(player_settlements)
                #if "cities" not in state:
                #    state["cities"] = {}
                state["cities"][point] = player_id
                state["player_scores"][player_id] += 1  # Cities give additional points
                print(f"Player {player_id} upgraded settlement at point {point} to a city")
                return True
        
        elif action == "development_card":
            if dev_card_deck:
                card = dev_card_deck.pop()
                player_dev_cards[player_id][card] += 1
                print(f"Player {player_id} bought a {card} development card")
                
                # Victory point cards are revealed immediatelystate["largest_army_info"] = (new_largest, max_knights)
                if card == "victory_point":
                    state["player_scores"][player_id] += 1
                    
                return True
            else:
                print("No development cards left in deck")
        
        return False
    
    def get_player_action(player, state):
        possible_actions = ["road", "settlement", "city", "development_card", "play_knight", "end_turn"]
        
        # Prioritize based on affordability and strategy
        priorities = []
        
        # Building priorities
        if can_afford(player, "settlement"):
            priorities.append("settlement")
        if can_afford(player, "city"):
            priorities.append("city")
        if can_afford(player, "road"):
            priorities.append("road")
        if can_afford(player, "development_card") and dev_card_deck:
            priorities.append("development_card")
        
        # Development card priorities
        if player_dev_cards[player]["knight"] > 0:
            # Play knight if we're close to largest army or need to move robber
            if knights_played[player] >= 2 or random.random() < 0.3:
                priorities.insert(0, "play_knight")  # High priority
        
        # 70% chance to do something if possible, 30% chance to end turn
        if priorities and random.random() < 0.7:
            return random.choice(priorities)
        else:
            return "end_turn"
    
    def smart_trade(player, state):
        """Execute smart trading decisions"""
        inv = inventories[player]
        
        # Find what we need for priority builds
        for build_type in ["settlement", "city", "development_card", "road"]:
            cost = building_costs[build_type]
            missing = {}
            
            for res, needed in cost.items():
                if inv.get(res, 0) < needed:
                    missing[res] = needed - inv.get(res, 0)
            
            if missing:
                # Try to trade excess resources
                excess_resources = {res: qty for res, qty in inv.items() if qty > 3}
                
                for give_res, qty in excess_resources.items():
                    ratio = get_trade_ratio(player, give_res, state)
                    if qty >= ratio:
                        get_res = random.choice(list(missing.keys()))
                        if execute_trade(player, give_res, get_res, state):
                            return True
        return False

    rolling_prob = {i: probability_sum_equals(i, n_die, die_sides) for i in range(n_die, n_die * die_sides + 1)}
    print(state['points_to_nodes'])
    starting_point_attractiveness = {
        point_id: sum(
            probability_sum_equals(grid[row, col, 0], n_die, die_sides)
            for row, col in state['points_to_nodes'][point_id]  # distinct
        )
        for point_id in range(num_vertices)
    }

    # Main game loop
    turn = 0
    
    print("=== GAME START ===")
    print(f"Players: {players}")
    print(f"Board size: {game_shape}")
    
    while not is_game_over(state, turn):
        print(f'\n=== TURN {turn + 1} ===')
        
        for player in range(num_players):
            print(f"\n--- Player {player}'s turn ---")
            
            # 1. Roll dice
            roll_result = roll()
            print(f"Rolled: {roll_result}")
            
            # 2. Handle special cases
            if roll_result == 7:
                # All players with >7 cards discard half
                for p in range(num_players):
                    discard_excess_cards(p)
                # Current player moves robber
                handle_robber(player, state)
            else:
                # Distribute resources
                distribute_resources(roll_result)
            
            # Show inventories
            for p in range(num_players):
                total = sum(inventories[p].values())
                print(f"Player {p}: {total} cards - {inventories[p]}")
            
            # 3. Trading phase
            if smart_trade(player, state):
                print(f"Player {player} made a trade")
            
            # 4. Building phase
            actions_taken = 0
            max_actions = 3  # Limit actions per turn
            
            while actions_taken < max_actions:
                action = get_player_action(player, state)
                
                if action == "end_turn":
                    break
                
                if action in building_costs and can_afford(player, action):
                    pay_resources(player, action)
                    if player_action(state, player, action):
                        actions_taken += 1
                        
                        # Check for longest road after building a road
                        if action == "road":
                            update_longest_road(state)
                            
                    else:
                        # Refund if action failed
                        cost = building_costs[action]
                        for res, qty in cost.items():
                            inventories[player][res] += qty
                else:
                    # Try to play development cards
                    if action == "play_knight" and player_dev_cards[player]["knight"] > 0:
                        if play_knight_card(player, state):
                            actions_taken += 1
                    else:
                        break
            
            # Check win condition
            total_score = calculate_total_score(player, state)
            if total_score >= victory_points_to_win:
                print(f"\n🎉 PLAYER {player} WINS! 🎉")
                print(f"Final score: {total_score}")
                print(f"Breakdown:")
                
                # Fixed calculation
                all_buildings = state['player_positions'][player]
                cities = [pt for pt in all_buildings if pt in state.get('cities', {})]
                settlements = [pt for pt in all_buildings if pt not in state.get('cities', {})]
                
                # Correct display - no double subtraction
                print(f"  - Settlements: {len(settlements)} points")  # 1 point each
                print(f"  - Cities: {len(cities)} points")            # 2 points each (but only 1 additional since they upgrade settlements)
                print(f"  - Victory Point cards: {player_dev_cards[player]['victory_point']}")
                
                if state.get("longest_road_info", (None, 0))[0] == player:
                    print(f"  - Longest Road: 2 points")
                if state.get("largest_army_info", (None, 0))[0] == player:
                    print(f"  - Largest Army: 2 points")
                
                return grid, state
        
        turn += 1
    
    # Game ended without winner
    winner = max(range(players), key=lambda p: calculate_total_score(p, state))
    winner_score = calculate_total_score(winner, state)
    
    print(f"\nGame ended after {turn} turns")
    print(f"Winner: Player {winner} with {winner_score} points")
    print("\nFinal scores:")
    for p in range(num_players):
        total = calculate_total_score(p, state)
        breakdown = []
        breakdown.append(f"Buildings: {len(state['player_positions'][p])}")
        if player_dev_cards[p]['victory_point'] > 0:
            breakdown.append(f"VP Cards: {player_dev_cards[p]['victory_point']}")
        # Only show "Longest Road: 2" if a player actually holds it (not if it's None)
        longest_road_player = state.get("longest_road_info", (None, 0))[0]
        if longest_road_player is not None and longest_road_player == p:
            breakdown.append("Longest Road: 2")
        largest_army_player = state.get("largest_army_info", (None, 0))[0]
        if largest_army_player is not None and largest_army_player == p:
            breakdown.append("Largest Army: 2")
        print(f"  Player {p}: {total} points ({', '.join(breakdown)})")
    
    return grid, state

In [6]:
class CatanConfig:
    def __init__(self, num_players=3, n_die=2, die_sides=6, board_size=(5, 5), max_turns=100, victory_points_to_win=10):
        self.num_players = num_players
        self.n_die = n_die
        self.die_sides = die_sides
        self.board_size = board_size
        self.max_turns = max_turns
        self.victory_points_to_win = victory_points_to_win

In [7]:
config = CatanConfig(
    num_players=3,
    n_die=2,
    die_sides=6,
    board_size=(5, 5),
    max_turns=500,
    victory_points_to_win=10
)

grid, final_state = game_loop(config, seed=1200)

longest_road = final_state.get("longest_road_info", (None, 0))
largest_army = final_state.get("largest_army_info", (None, 0))

print("\n=== GAME SUMMARY ===")
print("Final player scores:", final_state["player_scores"])
print("Total settlements built:", len(final_state["point_owner"]) - len(final_state.get("cities", {})))
print("Total roads built:", len(final_state.get("edge_owner", {})))
print("Total cities built:", len(final_state.get("cities", {})))
if longest_road[0] is not None:
    print(f"Longest road: Player {longest_road[0]} with length {longest_road[1]}")
if largest_army[0] is not None:
    print(f"Largest army: Player {largest_army[0]} with size {largest_army[1]}")
final_state = plot_state_and_index(grid, final_state)

UnboundLocalError: cannot access local variable 'state' where it is not associated with a value

In [ ]:
print(final_state['longest_road_info'])

(1, 8)
